# Baneco to YNAB Converter

This notebook converts Baneco CSV statements to YNAB CSV format.

Upload your Baneco CSV file and run the cells to generate `ynab.csv`.

In [ ]:
import csv
import io
from ipywidgets import FileUpload, Button, Output, VBox
from IPython.display import display, HTML
import base64

In [ ]:
# Month translation dictionary
months_dict = {
    "ene": "01",
    "feb": "02",
    "mar": "03",
    "abr": "04",
    "may": "05",
    "jun": "06",
    "jul": "07",
    "ago": "08",
    "sep": "09",
    "oct": "10",
    "nov": "11",
    "dic": "12",
}

In [ ]:
def convert_baneco_to_ynab(file_content):
    """Convert Baneco CSV content to YNAB CSV format."""
    
    # Decode the uploaded file content
    content = file_content.decode('utf-8')
    input_file = io.StringIO(content)
    output = io.StringIO()
    
    reader = csv.DictReader(input_file)
    writer = csv.writer(output)
    writer.writerow(["Date", "Payee", "Category", "Memo", "Outflow", "Inflow"])
    
    row_count = 0
    for row in reader:
        fecha = row["Fecha"].lower().split("/")
        mes = months_dict[fecha[1]]
        fecha_ynab = fecha[0] + "/" + mes + "/" + fecha[2]
        
        # Use Nota column if available, otherwise use Transaccion column
        memo = row["Nota"] if row["Nota"].strip() else row["Transaccion"]
        
        monto = float(row["Monto"].replace(",", "."))
        
        if monto < 0:
            outflow = str(-monto)
            inflow = ""
        else:
            outflow = ""
            inflow = str(monto)
        
        payee = ""
        if "YANGO" in memo.upper():
            payee = "Yango"
        elif "UBER" in memo.upper():
            payee = "Uber"
        elif "FIDALGA" in memo.upper():
            payee = "Fidalga"
        elif "FMC" in memo.upper():
            payee = "Farmacorp"
        elif "NETFLIX" in memo.upper():
            payee = "Netflix"
        elif "TIENDASTRESB" in memo.upper():
            payee = "Tiendas Tres B"
        elif "HIPERMAXI" in memo.upper():
            payee = "Hipermaxi"
        elif "ICNORTE" in memo.upper():
            payee = "IC Norte"
        elif "FARMACIACHAVEZ" in memo.upper():
            payee = "Farmacia Chávez"
        elif "CHATGPT" in memo.upper():
            payee = "ChatGPT"
        elif "GITHUB" in memo.upper():
            payee = "GitHub"
        
        writer.writerow([fecha_ynab, payee, "", memo, outflow, inflow])
        row_count += 1
    
    return output.getvalue(), row_count

In [ ]:
def create_download_link(content, filename):
    """Create a download link for the generated file."""
    b64 = base64.b64encode(content.encode()).decode()
    return HTML(f'<a download="{filename}" href="data:text/csv;base64,{b64}" target="_blank">Download {filename}</a>')

## Upload and Convert

Click the button below to upload your Baneco CSV file, then click "Convert to YNAB" to process it.

In [ ]:
# Create widgets
upload_widget = FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload CSV'
)

convert_button = Button(
    description='Convert to YNAB',
    button_style='success',
    disabled=True
)

output_widget = Output()

def on_upload_change(change):
    """Enable convert button when file is uploaded."""
    convert_button.disabled = len(upload_widget.value) == 0

def on_convert_click(b):
    """Process the uploaded file and generate YNAB CSV."""
    output_widget.clear_output()
    
    with output_widget:
        if not upload_widget.value:
            print("Please upload a file first!")
            return
        
        try:
            # Get the uploaded file
            uploaded_file = list(upload_widget.value.values())[0]
            filename = list(upload_widget.value.keys())[0]
            
            print(f"Processing {filename}...")
            
            # Convert the file
            ynab_content, row_count = convert_baneco_to_ynab(uploaded_file['content'])
            
            # Save to file
            with open('ynab.csv', 'w') as f:
                f.write(ynab_content)
            
            print(f"✓ Conversion complete! Processed {row_count} transactions.")
            print(f"✓ File saved as: ynab.csv")
            print("\nYou can also download it directly:")
            display(create_download_link(ynab_content, 'ynab.csv'))
            
        except Exception as e:
            print(f"Error during conversion: {str(e)}")
            import traceback
            traceback.print_exc()

upload_widget.observe(on_upload_change, names='value')
convert_button.on_click(on_convert_click)

# Display the interface
display(VBox([upload_widget, convert_button, output_widget]))